# Build a Crew to Tailor Job Applications ✨

## Introduction 
In this lesson, you will `build a multi-agent system using the crewAI` library. The system will help tailor job applications by creating a team of agents, each with a specific role in the job application process. This notebook will guide you through the process of setting up and running the system.

**Note** If you're running this notebook on your own machine, you can install the following:
```Py
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
# Suppress warnings for a cleaner output
import warnings
warnings.filterwarnings('ignore')

### Import libraries, APIs and LLM

In [ ]:
# Importing necessary classes from the crewai library
from crewai import Agent, Task, Crew

**Note** _Agent_ represents an agent with specific roles and goals. _Task_ represents a task that the agent needs to perform. _Crew_ represents a group of agents working together on tasks.    

In [ ]:
# Import the required libraries
import os
from utils import get_openai_api_key, get_serper_api_key

# Fetch the OpenAI API key using a utility function and store it in a variable
openai_api_key = get_openai_api_key()

# Set the environment variable for the OpenAI model name to 'gpt-4-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

# Fetch the Serper API key using a utility function and set it as an environment variable
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [ ]:
# Import the tools from the crewai_tools library.
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

# Initialize tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb:   0%|          | 0/1 [00:02<?, ?it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))

## Creating Agents

We will now create the agents that will perform various tasks in the job application process.

- Agent 1: Tech Job Researcher  
This agent analyzes job postings to extract key requirements.

In [5]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

- Agent 2: Personal Profiler for Engineers
This agent compiles a detailed personal and professional profile for job applicants.

In [6]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

- Agent 3: Resume Strategist for Engineers  
This agent refines resumes to highlight the most relevant skills and experiences.

In [7]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

- Agent 4: Engineering Interview Preparer  
This agent creates interview questions and talking points based on the resume and job requirements.

In [8]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

We will now create the tasks assigned to each agent.

- Task for Researcher Agent: Extract Job Requirements  
This task analyzes the job posting URL to extract key requirements.

In [9]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

- Task for Profiler Agent: Compile Comprehensive Profile  
This task compiles a detailed personal and professional profile using provided URLs and personal write-up.

In [10]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- Task for Resume Strategist Agent: Align Resume with Job Requirements  
This task tailors the resume to highlight the most relevant areas based on the profile and job requirements.

In [11]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

**Note**  You can pass a list of tasks as `context` to a task. The task then takes into account the output of those tasks in its execution. However, the task will not run until it has the output(s) from those tasks.

- Task for Interview Preparer Agent: Develop Interview Materials  
This task creates a set of potential interview questions and talking points based on the tailored resume and job requirements.

In [12]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

We will now create the crew that includes all agents and tasks.

In [13]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew and run it.

In [14]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
# This execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

Exception in thread Thread-9 (_execute):
Traceback (most recent call last):
  File "c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\Michela\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\site-packages\crewai\task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\site-packages\crewai\agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Michela\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\cha



> Entering new CrewAgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

- Dislplay the generated `tailored_resume.md` file.

In [16]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

./tailored_resume.md

- Dislplay the generated `interview_materials.md` file.

In [17]:
display(Markdown("./interview_materials.md"))

./interview_materials.md

## Conclusion 
In this notebook, you learned how to build a multi-agent system using the crewAI library to tailor job applications. We walked through the steps of setting up the environment, importing necessary libraries and tools, creating specialized agents, and defining tasks for each agent.

The agents collaboratively worked to analyze a job posting, compile a comprehensive profile, tailor a resume, and develop interview materials. This demonstrates the power and flexibility of multi-agent systems in automating and enhancing the job application process.

By completing this lesson, you have gained practical experience in creating and managing multi-agent systems, which can be applied to various real-world scenarios beyond job applications.

## Try yourself 
Now that you have learned how to build and run a multi-agent system to tailor job applications, it's time to try it yourself! You can modify the parameters such as the job posting URL, GitHub profile URL, and personal write-up to see how the system adapts to different inputs. Follow the steps below to customize and run the system with your own data.

Step 1: Set Your Inputs
Replace the example inputs with your own data.

In [ ]:
# Set your own inputs for the job application crew
job_application_inputs = {
    'job_posting_url': 'YOUR_JOB_POSTING_URL_HERE',
    'github_url': 'YOUR_GITHUB_PROFILE_URL_HERE',
    'personal_writeup': """YOUR_PERSONAL_WRITEUP_HERE"""
}

Step 2: Run the Crew
Execute the cell to run the crew with your customized inputs.

In [ ]:
# This execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

Step 3: Display the Results
View the generated tailored_resume.md and interview_materials.md files to see how the system has tailored the job application materials based on your inputs.

In [ ]:
# Display the generated tailored resume
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Display the generated interview materials
display(Markdown("./interview_materials.md"))

Example Inputs  
Here are some example inputs you can use to test the system:

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/ExampleCompany/12345',
    'github_url': 'https://github.com/exampleuser',
    'personal_writeup': """John Doe is a seasoned software engineer with over 10 years of experience in full-stack development. He has a strong background in AI and machine learning, and has led multiple successful projects at top tech companies. John is looking for a challenging role where he can leverage his skills to drive innovation and growth."""
}

Experiment and Explore  

Feel free to experiment with different job postings, GitHub profiles, and personal write-ups to see how the multi-agent system performs. This is a great way to understand the capabilities and flexibility of the system.  

Happy experimenting!